In [ ]:
!pip3 install mlflow
!pip install scikit-learn
!pip install flaml
!pip install "flaml[automl]"
!pip install flaml[automl]
%pip install flaml[automl] matplotlib openml
!pip install evidently
!pip install torch
!pip install --upgrade typing_extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.8/118.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%restart_python 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from flaml import AutoML
from sklearn.metrics import fbeta_score, f1_score, recall_score, precision_score, make_scorer, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
import requests
import json
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset
from evidently import ColumnMapping
from sklearn.metrics import accuracy_score, f1_score, fbeta_score, precision_score, recall_score
import torch
from evidently.ui.workspace.cloud import CloudWorkspace
from evidently.ui.dashboards import DashboardPanelPlot, ReportFilter, PanelValue, TestFilter, TestSuitePanelType, DashboardPanelTestSuite, PlotType
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Define input parameters for the notebook
dbutils.widgets.text("data_csv", "")  # For the CSV file path
dbutils.widgets.text("model_endpoint", "")  # For the model endpoint URL
dbutils.widgets.text("access_token", "")  # For the access token

# Retrieve the values of the parameters
data_csv = dbutils.widgets.get("data_csv")
model_endpoint = dbutils.widgets.get("model_endpoint")
access_token = dbutils.widgets.get("access_token")

# Print the retrieved parameters (for debugging purposes)
print(f"Data CSV Path: {data_csv}")
print(f"Model Endpoint: {model_endpoint}")
print(f"Access Token: {access_token}")

In [ ]:
# /Workspace/Users/liu20@uchicago.edu/diabetes.csv
df = pd.read_csv(data_csv)
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [ ]:
df.shape

(70692, 22)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       70692 non-null  float64
 1   HighBP                70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   Stroke                70692 non-null  float64
 7   HeartDiseaseorAttack  70692 non-null  float64
 8   PhysActivity          70692 non-null  float64
 9   Fruits                70692 non-null  float64
 10  Veggies               70692 non-null  float64
 11  HvyAlcoholConsump     70692 non-null  float64
 12  AnyHealthcare         70692 non-null  float64
 13  NoDocbcCost           70692 non-null  float64
 14  GenHlth               70692 non-null  float64
 15  MentHlth           

In [ ]:
df["Diabetes_binary"].value_counts()

0.0    35346
1.0    35346
Name: Diabetes_binary, dtype: int64

#### Train/test split with stratify technique

In [ ]:
X = df.drop(columns=['Diabetes_binary'])  
y = df['Diabetes_binary']  

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.1,  # 10% for test
    stratify=y,     # Stratify by the target variable
    random_state=42 # Ensure reproducibility
)

train_df = X_train.copy()
train_df['Diabetes_binary'] = y_train

test_df = X_test.copy()
test_df['Diabetes_binary'] = y_test

# Check class balance in train and test sets
print("Train class distribution:\n", train_df['Diabetes_binary'].value_counts(normalize=True))
print("Test class distribution:\n", test_df['Diabetes_binary'].value_counts(normalize=True))

Train class distribution:
 0.0    0.5
1.0    0.5
Name: Diabetes_binary, dtype: float64
Test class distribution:
 1.0    0.5
0.0    0.5
Name: Diabetes_binary, dtype: float64


#### Model inference on test set & model monitoring with Evidently AI

In [ ]:
X_test

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
65435,1.0,1.0,1.0,35.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,5.0,0.0,0.0,6.0,6.0,7.0
35694,1.0,1.0,1.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,30.0,30.0,1.0,1.0,5.0,6.0,1.0
66509,1.0,1.0,1.0,36.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,4.0,0.0,7.0,1.0,0.0,8.0,6.0,5.0
18354,1.0,0.0,1.0,34.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,4.0,30.0,30.0,1.0,0.0,11.0,6.0,8.0
58339,1.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,13.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44438,1.0,0.0,1.0,30.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,8.0,4.0,6.0
47342,1.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,4.0,4.0
2151,1.0,0.0,1.0,19.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,5.0,0.0,0.0,9.0,6.0,7.0
24442,0.0,1.0,1.0,27.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,3.0,1.0,0.0,0.0,13.0,4.0,6.0


In [ ]:
# Convert X_test to payload format
payload = {
    "dataframe_split": {
        "columns": list(X_test.columns),  # Feature names
        "data": X_test.values.tolist()    # Feature values (row-wise)
    }
}

headers = {
    "Authorization": f"Bearer {access_token}",  
    "Content-Type": "application/json"
}


In [ ]:
# Send the POST request to the endpoint
response = requests.post(model_endpoint, headers=headers, data=json.dumps(payload))

if response.status_code == 200:
    # Extract predictions from the response
    predictions = response.json().get("predictions", [])
    predictions = pd.Series(predictions, name="Predicted")  # Convert to Pandas Series

    # Prepare the Actual and Predicted Data
    results = pd.DataFrame({
        "Actual": y_test.reset_index(drop=True),  # Reset index for alignment
        "Predicted": predictions
    })

    print("Predictions DataFrame:")
    print(results)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    f2 = fbeta_score(y_test, predictions, beta=2)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"F2 Score: {f2:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # -------------------------
    # Evidently Monitoring Setup
    # -------------------------

    # Prepare Reference (Training) Data
    reference_data = pd.DataFrame(X_train, columns=X_train.columns)
    reference_data["Target"] = y_train.reset_index(drop=True)  # Add target column to reference data

    # Prepare Current (Test) Data
    current_data = pd.DataFrame(X_test, columns=X_test.columns)
    current_data["Target"] = predictions  # Add predictions as the target column for monitoring

    # Column Mapping
    column_mapping = ColumnMapping(
        target="Target",  # Target column (Actual for Reference, Predicted for Current)
        prediction=None,  # No explicit prediction column in Evidently for this report
        numerical_features=X_test.columns.tolist()  # List of numerical feature columns
    )

    # Generate Evidently Reports
    drift_report = Report(metrics=[DataDriftPreset(), TargetDriftPreset()])
    drift_report.run(reference_data=reference_data, current_data=current_data, column_mapping=column_mapping)

    # Save Evidently Reports
    drift_report.save_html("before_swapping_drift_report.html")

    print("Drift report saved: before_swapping_drift_report.html")

else:
    print("Error Response:", response.json())


Predictions DataFrame:
      Actual  Predicted
0        1.0          1
1        1.0          1
2        1.0          1
3        0.0          1
4        1.0          1
...      ...        ...
7065     1.0          1
7066     1.0          1
7067     0.0          0
7068     0.0          1
7069     0.0          1

[7070 rows x 2 columns]
Accuracy: 0.7496
F1 Score: 0.7627
F2 Score: 0.7872
Precision: 0.7250
Recall: 0.8045
Drift report saved: before_swapping_drift_report.html


#### Log model performance on test set before swapping features

In [ ]:
def plot_and_log_confusion_matrix(y_true, y_pred, title, filename, prefix=""):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
    disp.plot(cmap="Blues", values_format="d")
    plt.title(title)
    plt.tight_layout()

    plt.savefig(filename)
    plt.close()

    # Log the plot as an artifact
    mlflow.log_artifact(filename, artifact_path=f"plots/{prefix}")
    print(f"Confusion matrix saved and logged to MLflow: {filename}")

# mlflow.end_run()

# Start a single MLflow run for both before and after swapping
with mlflow.start_run(run_name="Model Eval Before Swapping"):

    # ---- BEFORE SWAPPING ----
    print("Logging BEFORE swapping metrics and confusion matrix...")

    # Calculate metrics
    metrics_before = {
        "Accuracy": accuracy_score(y_test, predictions),
        "F1_Score": f1_score(y_test, predictions),
        "F2_Score": fbeta_score(y_test, predictions, beta=2),
        "Precision": precision_score(y_test, predictions),
        "Recall": recall_score(y_test, predictions)
    }

    # Log metrics
    for key, value in metrics_before.items():
        mlflow.log_metric(f"before_{key}", value)

    # Create and log the confusion matrix
    plot_and_log_confusion_matrix(
        y_true=y_test.reset_index(drop=True),
        y_pred=predictions,
        title="Confusion Matrix Before Swapping",
        filename="before_swapping_confusion_matrix.png",
        prefix="before"
    )

    # Log the monitoring report generated previously
    mlflow.log_artifact("before_swapping_drift_report.html", artifact_path="evidently_reports")

    print("Metrics and confusion matrix for BEFORE swapping logged.")
    

Logging BEFORE swapping metrics and confusion matrix...
Confusion matrix saved and logged to MLflow: before_swapping_confusion_matrix.png
Metrics and confusion matrix for BEFORE swapping logged.
🏃 View run Model Eval Before Swapping at: https://dbc-cc567933-f83f.cloud.databricks.com/ml/experiments/1016961041466494/runs/87d987a79c8f41bf81c78258863bd85a
🧪 View experiment at: https://dbc-cc567933-f83f.cloud.databricks.com/ml/experiments/1016961041466494


#### Swap features and evaluate model performance again

In [ ]:
# Perform feature swapping and monitoring
with mlflow.start_run(run_name="Model Eval After Swapping"):
    print("Performing feature swapping and logging metrics and confusion matrix...")

    # ---- SWAP FEATURES ----
    X_test_swapped = X_test.copy()
    X_test_swapped["MentHlth"], X_test_swapped["BMI"] = X_test_swapped["BMI"], X_test_swapped["MentHlth"]

    # Prepare payload with swapped features
    payload_swapped = {
        "dataframe_split": {
            "columns": list(X_test_swapped.columns),
            "data": X_test_swapped.values.tolist()
        }
    }

    # Send inference request with swapped features
    response_swapped = requests.post(model_endpoint, headers=headers, data=json.dumps(payload_swapped))

    if response_swapped.status_code == 200:
        # Process predictions
        predictions_swapped = response_swapped.json().get("predictions", [])
        predictions_swapped = pd.Series(predictions_swapped, name="Predicted")

        # ---- METRICS AFTER SWAPPING ----
        metrics_after = {
            "Accuracy": accuracy_score(y_test, predictions_swapped),
            "F1_Score": f1_score(y_test, predictions_swapped),
            "F2_Score": fbeta_score(y_test, predictions_swapped, beta=2),
            "Precision": precision_score(y_test, predictions_swapped),
            "Recall": recall_score(y_test, predictions_swapped)
        }

        # Log metrics
        for key, value in metrics_after.items():
            mlflow.log_metric(f"after_{key}", value)

        # Create and log the confusion matrix
        plot_and_log_confusion_matrix(
            y_true=y_test.reset_index(drop=True),
            y_pred=predictions_swapped,
            title="Confusion Matrix After Swapping",
            filename="after_swapping_confusion_matrix.png",
            prefix="after"
        )

        print("Metrics and confusion matrix for AFTER swapping logged.")

        # ---- EVIDENTLY MONITORING ----
        print("Generating Evidently reports...")

        # Prepare Reference (Training) Data
        reference_data = pd.DataFrame(X_train, columns=X_train.columns)
        reference_data["Target"] = y_train.reset_index(drop=True)

        # Prepare Current (Swapped Test) Data
        current_data = pd.DataFrame(X_test_swapped, columns=X_test_swapped.columns)
        current_data["Target"] = predictions_swapped.reset_index(drop=True)

        # Column Mapping for Evidently
        column_mapping = ColumnMapping(
            target="Target",  # Target column (Actual for Reference, Predicted for Current)
            prediction=None,  # No explicit prediction column
            numerical_features=X_test.columns.tolist()  # Numerical features
        )

        # Generate Evidently Reports
        drift_report = Report(metrics=[DataDriftPreset(), TargetDriftPreset()])
        drift_report.run(reference_data=reference_data, current_data=current_data, column_mapping=column_mapping)

        # Save Evidently Reports
        drift_report.save_html("after_swapping_drift_report.html")

        # Log Drift Report to MLflow
        mlflow.log_artifact("after_swapping_drift_report.html", artifact_path="evidently_reports")
        print("Drift report saved and logged: after_swapping_drift_report.html")

    else:
        print("Error during inference with swapped features:", response_swapped.json())


Performing feature swapping and logging metrics and confusion matrix...
Confusion matrix saved and logged to MLflow: after_swapping_confusion_matrix.png
Metrics and confusion matrix for AFTER swapping logged.
Generating Evidently reports...
Drift report saved and logged: after_swapping_drift_report.html
🏃 View run Model Eval After Swapping at: https://dbc-cc567933-f83f.cloud.databricks.com/ml/experiments/1016961041466494/runs/0dcaedb7def34ded8465389374bd08cb
🧪 View experiment at: https://dbc-cc567933-f83f.cloud.databricks.com/ml/experiments/1016961041466494
